In [5]:
import os.path
from googleapiclient.discovery import build
from google.oauth2 import service_account
import pandas as pd


SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
BASE_DIR = os.path.dirname(os.path.abspath("scripts.py"))
SERVICE_ACCOUNT_FILE = os.path.join(BASE_DIR, 'local-talent-364913-0febf38e0456.json')

credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# The ID and range of a sample spreadsheet.

SAMPLE_SPREADSHEET_ID = '1oYSVwlNr2NZSB6i2pNHyKN0aW34Y50jldQgEKHUJiVw'
SAMPLE_RANGE_NAME = 'Real_Data'


service = build('sheets', 'v4', credentials=credentials)

sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME).execute()["values"][2:]
reasons = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range="Reasons").execute()["values"][2:]

In [9]:
# Fill names on each row

j = ''
for i in result:
    if i[0] == '':
        i[0] = j
    j = i [0]

In [13]:
df = pd.DataFrame(result)
df = df[df[1] != '']
df["sum_of_pluses"] = df[[2, 3, 4, 5]].T.apply(lambda x: x.str.count("\+")).sum()
df.drop([2, 3, 4, 5], axis=1, inplace=True)
df.rename({0: 'student', 1: "reason"}, axis=1, inplace=True)
df

,student,reason,sum_of_pluses
0,Гриб Анастасия,Предложил эксперта для мероприятия в школе и с...,0
1,Гриб Анастасия,Разместил пост/сторис в соц.сети с работой или...,0
2,Гриб Анастасия,Участие в мероприятиях общих (просто участие ж...,0
3,Гриб Анастасия,Участие в метаворкшопе или спецворкшопе (зелен...,1
4,Гриб Анастасия,Участие в консультации (зеленая карточка),1
5,Гриб Анастасия,Пришел на онбординг,2
17,Марченко Анна,Предложил эксперта для мероприятия в школе и с...,0
18,Марченко Анна,Разместил пост/сторис в соц.сети с работой или...,2
19,Марченко Анна,Участие в мероприятиях общих (просто участие ж...,0
20,Марченко Анна,Участие в метаворкшопе или спецворкшопе (зелен...,0


In [14]:
reasons_df = pd.DataFrame(reasons).drop([2, 3, 4], axis=1)
reasons_df.drop(reasons_df[reasons_df[1] == ""].index, inplace=True)

In [15]:
reasons_df.rename(columns={0: "reason", 1: "amount"}, inplace=True)
reasons_df.amount = reasons_df.amount.astype(int)

In [16]:
df = pd.merge(df, reasons_df)

In [18]:
df["to_add"] = df.sum_of_pluses * df["amount"].astype(int)

In [21]:
df[df.sum_of_pluses != 0]

,student,reason,sum_of_pluses,amount,to_add
3,Нижник Даниил,Предложил эксперта для мероприятия в школе и с...,1,40,40
6,Марченко Анна,Разместил пост/сторис в соц.сети с работой или...,2,5,10
8,Нижник Даниил,Разместил пост/сторис в соц.сети с работой или...,1,5,5
12,Крюкова ксения,Участие в мероприятиях общих (просто участие ж...,2,20,40
13,Нижник Даниил,Участие в мероприятиях общих (просто участие ж...,1,20,20
15,Гриб Анастасия,Участие в метаворкшопе или спецворкшопе (зелен...,1,20,20
18,Нижник Даниил,Участие в метаворкшопе или спецворкшопе (зелен...,2,20,40
20,Гриб Анастасия,Участие в консультации (зеленая карточка),1,20,20
21,Марченко Анна,Участие в консультации (зеленая карточка),1,20,20
25,Гриб Анастасия,Пришел на онбординг,2,10,20


In [22]:
df.groupby("student").to_add.sum()

student
Гриб Анастасия         60
Крюкова ксения         40
Марченко Анна          40
Нижник Даниил         125
Хизетль Анастасия      10
Name: to_add, dtype: int64

In [46]:
logs_df = pd.DataFrame(columns=['student', 'reason', 'amount', "date"])

In [47]:
from datetime import datetime
for row in df[df.sum_of_pluses != 0].values:
    for i in range(row[2]): 
        logs_df = logs_df.append({"student": row[0], "reason": row[1], "amount": row[3], "date": datetime.today()}, ignore_index=True)
logs_df.drop(1, inplace=True)
logs_df.reset_index(drop=True, inplace=True)
logs_df

/var/folders/41/wrrf18wx6v11s79l8m6f70fh0000gp/T/ipykernel_5742/2252156813.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  logs_df = logs_df.append({"student": row[0], "reason": row[1], "amount": row[3], "date": datetime.today()}, ignore_index=True)
/var/folders/41/wrrf18wx6v11s79l8m6f70fh0000gp/T/ipykernel_5742/2252156813.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  logs_df = logs_df.append({"student": row[0], "reason": row[1], "amount": row[3], "date": datetime.today()}, ignore_index=True)
/var/folders/41/wrrf18wx6v11s79l8m6f70fh0000gp/T/ipykernel_5742/2252156813.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  logs_df = logs_df.append({"student": row[0], "reason": row[1], "amount": row[3], "date": datetime.

,student,reason,amount,date
0,Нижник Даниил,Предложил эксперта для мероприятия в школе и с...,40,2022-10-18 16:40:30.521215
1,Марченко Анна,Разместил пост/сторис в соц.сети с работой или...,5,2022-10-18 16:40:30.526714
2,Нижник Даниил,Разместил пост/сторис в соц.сети с работой или...,5,2022-10-18 16:40:30.529076
3,Крюкова ксения,Участие в мероприятиях общих (просто участие ж...,20,2022-10-18 16:40:30.531443
4,Крюкова ксения,Участие в мероприятиях общих (просто участие ж...,20,2022-10-18 16:40:30.533438
5,Нижник Даниил,Участие в мероприятиях общих (просто участие ж...,20,2022-10-18 16:40:30.535703
6,Гриб Анастасия,Участие в метаворкшопе или спецворкшопе (зелен...,20,2022-10-18 16:40:30.538958
7,Нижник Даниил,Участие в метаворкшопе или спецворкшопе (зелен...,20,2022-10-18 16:40:30.541748
8,Нижник Даниил,Участие в метаворкшопе или спецворкшопе (зелен...,20,2022-10-18 16:40:30.544183
9,Гриб Анастасия,Участие в консультации (зеленая карточка),20,2022-10-18 16:40:30.546446


In [44]:
#TODO: parse logs from sheets

past_logs_df = logs_df.drop([2, 6, 7, 9])
past_logs_df.reset_index(inplace=True, drop=True)
past_logs_df

,student,reason,amount,date
0,Нижник Даниил,Предложил эксперта для мероприятия в школе и с...,40,2022-10-18 16:39:07.061244
1,Марченко Анна,Разместил пост/сторис в соц.сети с работой или...,5,2022-10-18 16:39:07.067115
2,Крюкова ксения,Участие в мероприятиях общих (просто участие ж...,20,2022-10-18 16:39:07.071420
3,Крюкова ксения,Участие в мероприятиях общих (просто участие ж...,20,2022-10-18 16:39:07.073615
4,Нижник Даниил,Участие в мероприятиях общих (просто участие ж...,20,2022-10-18 16:39:07.075999
5,Нижник Даниил,Участие в метаворкшопе или спецворкшопе (зелен...,20,2022-10-18 16:39:07.083411
6,Марченко Анна,Участие в консультации (зеленая карточка),20,2022-10-18 16:39:07.088054
7,Гриб Анастасия,Пришел на онбординг,10,2022-10-18 16:39:07.090152
8,Гриб Анастасия,Пришел на онбординг,10,2022-10-18 16:39:07.092174
9,Марченко Анна,Пришел на онбординг,10,2022-10-18 16:39:07.094268


In [48]:
logs_to_add = past_logs_df.drop('date', axis=1)\
                          .merge(logs_df.drop('date', axis=1), indicator = True, how='outer')\
                          .loc[lambda x : x['_merge']!='both']
logs_to_add.drop('_merge', axis=1, inplace=True)
logs_to_add

/var/folders/41/wrrf18wx6v11s79l8m6f70fh0000gp/T/ipykernel_5742/3637251113.py:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  logs_to_add = past_logs_df.drop('date', axis=1)\
/var/folders/41/wrrf18wx6v11s79l8m6f70fh0000gp/T/ipykernel_5742/3637251113.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  logs_to_add.drop('_merge', axis=1, inplace=True)


,student,reason,amount
20,Нижник Даниил,Разместил пост/сторис в соц.сети с работой или...,5
21,Гриб Анастасия,Участие в метаворкшопе или спецворкшопе (зелен...,20
22,Гриб Анастасия,Участие в консультации (зеленая карточка),20


In [51]:
new_logs_df = pd.concat([past_logs_df, logs_to_add])
new_logs_df.date.fillna(datetime.today(), inplace=True)
new_logs_df.reset_index(drop=True)

,student,reason,amount,date
0,Нижник Даниил,Предложил эксперта для мероприятия в школе и с...,40,2022-10-18 16:39:07.061244
1,Марченко Анна,Разместил пост/сторис в соц.сети с работой или...,5,2022-10-18 16:39:07.067115
2,Крюкова ксения,Участие в мероприятиях общих (просто участие ж...,20,2022-10-18 16:39:07.071420
3,Крюкова ксения,Участие в мероприятиях общих (просто участие ж...,20,2022-10-18 16:39:07.073615
4,Нижник Даниил,Участие в мероприятиях общих (просто участие ж...,20,2022-10-18 16:39:07.075999
5,Нижник Даниил,Участие в метаворкшопе или спецворкшопе (зелен...,20,2022-10-18 16:39:07.083411
6,Марченко Анна,Участие в консультации (зеленая карточка),20,2022-10-18 16:39:07.088054
7,Гриб Анастасия,Пришел на онбординг,10,2022-10-18 16:39:07.090152
8,Гриб Анастасия,Пришел на онбординг,10,2022-10-18 16:39:07.092174
9,Марченко Анна,Пришел на онбординг,10,2022-10-18 16:39:07.094268


In [52]:
new_logs_df.groupby('student').amount.sum()

student
Гриб Анастасия         60
Крюкова ксения         40
Марченко Анна          35
Нижник Даниил         105
Хизетль Анастасия      10
Name: amount, dtype: object